In [1]:
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df=pd.read_csv('precip_ET.csv',skiprows=16)
mean_air_temp_c = 273.15+df['T2M']
precip_ms = df['PRECTOTCORR']/86400000
incom_shor_rad=df['ALLSKY_SFC_SW_DWN']*1e6/86400
vap_pres=df['RH2M']*6.11*np.exp((17.27*df['T2M'])/(df['T2M']+237.3))
df

,YEAR,DOY,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,T2M,PRECTOTCORR,RH2M
0,2023,1,27.43,7.76,8.78,1.09,97.08
1,2023,2,30.43,4.21,12.42,1.66,95.93
2,2023,3,31.66,3.63,16.08,13.05,97.24
3,2023,4,27.60,8.96,13.38,20.30,87.15
4,2023,5,21.88,12.38,6.55,0.01,81.14
...,...,...,...,...,...,...,...
886,2025,156,33.19,21.89,22.96,4.48,78.24
887,2025,157,34.80,16.48,23.46,7.52,82.92
888,2025,158,35.21,21.00,22.89,11.29,85.04
889,2025,159,33.01,19.09,22.19,8.34,85.08


In [7]:
hf = h5py.File('daymet_data.h5', 'w')

In [8]:
hf.create_dataset('time [s]', data=np.arange(0,86400*len(precip_ms),86400))
hf.create_dataset('air temperature [K]', data=mean_air_temp_c)
hf.create_dataset('incoming shortwave radiation [W m^-2]', data=incom_shor_rad)
hf.create_dataset('vapor pressure air [Pa]', data=vap_pres)
hf.create_dataset('precipitation rain [m s^-1]', data=np.where(mean_air_temp_c >= 273.15, precip_ms, 0))
hf.create_dataset('precipitation snow [m SWE s^-1]', data=np.where(mean_air_temp_c < 273.15, precip_ms, 0))

<HDF5 dataset "precipitation snow [m SWE s^-1]": shape (891,), type "<f8">

In [9]:
hf.close()

In [12]:
###

In [24]:
STW2=pd.read_csv('../Field Data/Share_Chris/Forest_export_5.13.25/Forest_STW2water_level.csv')
STW2["Date & Time"]=pd.to_datetime(STW2["Date & Time"])
STW2=STW2.set_index('Date & Time')
STW2=STW2.resample('D').mean(numeric_only=True)
STW2

,Pressure[cmH2O],Temperature[°C],2:Spec.cond.[µS/cm],Pressure (cmH2O),Water level (cm)
Date & Time,,,,,
2024-03-26,1015.351014,12.274014,53.666667,1001.228704,14.122310
2024-03-27,1019.608156,12.685917,45.375000,1004.892969,14.715187
2024-03-28,1024.784094,12.056323,41.291667,1009.762587,15.021507
2024-03-29,1024.841729,11.856167,41.125000,1010.024479,14.817250
2024-03-30,1020.485656,12.739979,42.406250,1005.876128,14.609528
...,...,...,...,...,...
2025-03-03,1024.211625,5.822906,75.968750,1010.366584,13.845041
2025-03-04,1016.653219,7.202125,79.322917,1001.954031,14.699188
2025-03-05,1002.905427,9.099708,79.885417,988.231094,14.674333


In [35]:
hf = h5py.File('Site_1/Water_level.h5', 'w')
hf.create_dataset('time [s]', data=np.arange(38880000,38880000+86400*len(STW2),86400))
hf.create_dataset('Water_level [m]', data=STW2['Water level (cm)'].values/100)
hf.close()

In [37]:
GW=pd.read_csv('../Field Data/Share_Chris/Forest_export_5.13.25/Forest_NB4water_level.csv')
GW["Date & Time"]=pd.to_datetime(GW["Date & Time"])
GW=GW.set_index('Date & Time')
GW=GW.resample('D').mean(numeric_only=True)
GW=GW[GW.index > '2024-11-30']

In [38]:
hf=h5py.File('Site_1/GW_level.h5', 'w')
hf.create_dataset('time [s]', data=np.arange(60480000,60480000+86400*len(GW),86400))
hf.create_dataset('Water_level [m]', data=242.4902-3.66+GW['Water level (cm)'].values/100)
hf.close()